In [1]:
# process data and save to memory as variables, not storage
import sys
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import os
import numpy as np
from os import listdir
from os.path import isfile, join
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from PyPDF2 import PdfMerger
from tqdm import trange

2023-08-01 23:12:30.448545: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-01 23:12:30.480756: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-01 23:12:30.620759: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-01 23:12:30.621462: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-01 23:12:31.338266: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [2]:
wireplane='U'

In [3]:
# takes full raw data and extracts waveform of length nticks
def extract_wave(data, nticks=200):
    string = 'tck_'
    waveforms = []
    #Here I extract a column in each iteration and append to list
    for i in range(nticks):
        waveforms.append(data[string+str(i)].astype(np.int16))
    #convert to numpy ndarray
    waveforms = np.array(waveforms).astype(np.int16)
    #since raws and columns are inverted we need to transpose it
    return np.transpose(waveforms)

# takes full raw data and returns waveform of length nticks
# only keeps waves at a desired adc count 
def get_std_waveforms(data_noisy, data_clean, nticks=200, min_adc=5):
    #Extract and scale waveform data (passthrough rn)
    raw_waveforms_noisy = extract_wave(data_noisy, nticks)
    raw_waveforms_clean = extract_wave(data_clean, nticks)
    #print('before adc filter: ', raw_waveforms_noisy.shape, raw_waveforms_clean.shape)

    noisy_ = []
    clean_ = []

    for i, wave in enumerate(raw_waveforms_clean):
        if max(wave) >= min_adc:
            noisy_.append(raw_waveforms_noisy[i])
            clean_.append(wave)
    
    del raw_waveforms_noisy, raw_waveforms_clean

    noisy_ = np.array(noisy_)
    clean_ = np.array(clean_)

    #print('after adc filter: ', noisy_.shape, clean_.shape)
    #print(raw_waveforms) 
    #scaled_waveforms = waveform_scaler.fit_transform(raw_waveforms)
    return noisy_, clean_



In [4]:
import tarfile
import os

# nuCC
dir_noisy_nu_cc = '/home/vlian/Workspace/train_dune_lartpc_v2/nu_cc/'+wireplane+'/noisy_signal/'  # Directory to extract files
dir_clean_nu_cc = '/home/vlian/Workspace/train_dune_lartpc_v2/nu_cc/'+wireplane+'/clean_signal/'  # Directory to extract files

noisy_names_cc = os.listdir(dir_noisy_nu_cc)
noisy_names_cc = sorted(noisy_names_cc)


clean_names_cc = os.listdir(dir_clean_nu_cc)
clean_names_cc = sorted(clean_names_cc)

# nuES
dir_noisy_nu_es = '/home/vlian/Workspace/train_dune_lartpc_v2/nu_es/'+wireplane+'/noisy_signal/'  # Directory to extract files
dir_clean_nu_es = '/home/vlian/Workspace/train_dune_lartpc_v2/nu_es/'+wireplane+'/clean_signal/'  # Directory to extract files

noisy_names_es = os.listdir(dir_noisy_nu_es)
noisy_names_es = sorted(noisy_names_es)

clean_names_es = os.listdir(dir_clean_nu_es)
clean_names_es = sorted(clean_names_es)


In [5]:
print(len(noisy_names_cc), len(clean_names_cc))
print(len(noisy_names_es), len(clean_names_es))

52 52
188 188


In [6]:
print(noisy_names_cc[0], '---', clean_names_cc[0])
print(noisy_names_es[0], '---', clean_names_es[0])

snb-nucc-en0-U-signal-67877023-0-0.npy --- snb-nucc-en0-U-clnsig-67877023-0-0.npy
snb-nues-en0-U-signal-101992-0-0.npy --- snb-nues-en0-U-clnsig-101992-0-0.npy


### seperate by energy

In [7]:
def get_wave_by_ENRG(energy_idx, noisy_filenames, clean_filenames, interaction_type=True):
    if interaction_type:
        noisy_path = dir_noisy_nu_cc
        clean_path = dir_clean_nu_cc
    else:
        noisy_path = dir_noisy_nu_es
        clean_path = dir_clean_nu_es

    file_names_noisy = [file for file in noisy_filenames if 'en'+str(energy_idx) in file ]
    file_names_clean = [file for file in clean_filenames if 'en'+str(energy_idx) in file ]

    noisy_waveforms = np.empty((0, 200))
    clean_waveforms = np.empty((0, 200))

    for i, file_name in enumerate(file_names_noisy):
        noisy_file_path = os.path.join(noisy_path, file_name)
        clean_file_path = os.path.join(clean_path, file_names_clean[i])
        

        noisy = np.load(noisy_file_path)
        clean = np.load(clean_file_path)

        noisy_wf, clean_wf = get_std_waveforms(noisy, clean, nticks=200, min_adc=5)
        
        noisy_waveforms = np.concatenate((noisy_waveforms, noisy_wf))
        clean_waveforms = np.concatenate((clean_waveforms, clean_wf))

    return [noisy_waveforms, clean_waveforms]


In [8]:
all_waveforms_nu_CC = []
all_waveforms_nu_ES = []
for i in range(10):
    all_waveforms_nu_CC.append(get_wave_by_ENRG(i, noisy_names_cc, clean_names_cc))
    all_waveforms_nu_ES.append(get_wave_by_ENRG(i, noisy_names_es, clean_names_es, False))

In [9]:
for i, en in enumerate(all_waveforms_nu_CC):
    print('en',i,':', 'nu_CC',en[0].shape, en[1].shape,'---','nu_ES', all_waveforms_nu_ES[i][0].shape, all_waveforms_nu_ES[i][1].shape)

en 0 : nu_CC (4479, 200) (4479, 200) --- nu_ES (7080, 200) (7080, 200)
en 1 : nu_CC (4377, 200) (4377, 200) --- nu_ES (6475, 200) (6475, 200)
en 2 : nu_CC (4701, 200) (4701, 200) --- nu_ES (6353, 200) (6353, 200)
en 3 : nu_CC (4869, 200) (4869, 200) --- nu_ES (6268, 200) (6268, 200)
en 4 : nu_CC (4828, 200) (4828, 200) --- nu_ES (6346, 200) (6346, 200)
en 5 : nu_CC (5891, 200) (5891, 200) --- nu_ES (6525, 200) (6525, 200)
en 6 : nu_CC (5745, 200) (5745, 200) --- nu_ES (6588, 200) (6588, 200)
en 7 : nu_CC (5919, 200) (5919, 200) --- nu_ES (6169, 200) (6169, 200)
en 8 : nu_CC (5991, 200) (5991, 200) --- nu_ES (6860, 200) (6860, 200)
en 9 : nu_CC (6809, 200) (6809, 200) --- nu_ES (6474, 200) (6474, 200)


### Load Noise

In [10]:
noise_path = '/home/vlian/Workspace/train_dune_lartpc_v2/noise/'+wireplane+'/'
noise_filenames = sorted([f for f in listdir(noise_path) if (isfile(join(noise_path, f)) and wireplane in f)])
combined_noise = np.concatenate([np.load(noise_path+fname, mmap_mode='r') for fname in noise_filenames])

noise_waveforms = extract_wave(combined_noise)
roi_truth_noise = np.zeros(noise_waveforms.shape[0]) # for roi-finding
print(noise_waveforms.shape, roi_truth_noise.shape)

(100000, 200) (100000,)


### TEST

In [11]:
model_5_10_mean = np.load('../models_scales/mean_5_10' + wireplane + '_nu.npy')
model_5_10_std = np.load('../models_scales/scale_5_10' + wireplane + '_nu.npy')

model_5_15_mean = np.load('../models_scales/mean_5_15' + wireplane + '_nu.npy')
model_5_15_std = np.load('../models_scales/scale_5_15' + wireplane + '_nu.npy')

model_5_18_mean = np.load('../models_scales/mean_5_18' + wireplane + '_nu.npy')
model_5_18_std = np.load('../models_scales/scale_5_18' + wireplane + '_nu.npy')

model_60k_mean = np.load('../models_scales/mean_60k' + wireplane + '_nu.npy')
model_60k_std = np.load('../models_scales/scale_60k' + wireplane + '_nu.npy')

In [12]:
scalers = [[model_5_10_mean, model_5_10_std], [model_5_15_mean, model_5_15_std], 
           [model_5_18_mean, model_5_18_std], [model_60k_mean, model_60k_std]]

In [13]:
model_5_10 = load_model('../ROI_ar39_models/model_5_10' + wireplane + 'plane_nu_ROI.h5')

model_5_15 = load_model('../ROI_ar39_models/model_5_15' + wireplane + 'plane_nu_ROI.h5')

model_5_18 = load_model('../ROI_ar39_models/model_5_18' + wireplane + 'plane_nu_ROI.h5')

model_60k = load_model('../ROI_ar39_models/model_60k' + wireplane + 'plane_nu_ROI.h5')

In [14]:
models = [model_5_10, model_5_15, model_5_18, model_60k]

In [15]:
def eval_model(idx, cnn_min):
    model_idx = idx

    noise_scaled = (noise_waveforms-scalers[model_idx][0])/scalers[model_idx][1]
    infer = models[model_idx].predict(noise_scaled, verbose=0)
    
    return (len([i for i in infer if i > cnn_min])/len(infer))*100

In [ ]:
for i in range(4):
    print(round(eval_model(i, 0.999), 5), round(eval_model(i, 0.94), 5))

In [ ]:
def eval_model_en(en_group_waveforms, model_idx, cnn_min):

    waveforms_scaled = (en_group_waveforms-scalers[model_idx][0])/scalers[model_idx][1]
    infer = models[model_idx].predict(waveforms_scaled, verbose=0)
    
    return (len([i for i in infer if i > cnn_min])/len(infer))*100

In [ ]:
for i, signals_at_en in enumerate(all_waveforms_nu_ES):
    print('en:', i)
    for j in range(4):
        #print('    model:', j)
        print(round(eval_model_en(signals_at_en[0], j, 0.999), 2), round(eval_model_en(signals_at_en[0], j, 0.94), 2))
    print('--------------')

### Denoising Autoencoder

In [16]:
model_5_10_AE = load_model('../AE_ar39_models/model_5_10' + wireplane + 'plane_nu_AE.h5')

model_5_15_AE = load_model('../AE_ar39_models/model_5_15' + wireplane + 'plane_nu_AE.h5')

model_5_18_AE = load_model('../AE_ar39_models/model_5_18' + wireplane + 'plane_nu_AE.h5')

model_60k_AE = load_model('../AE_ar39_models/model_60k' + wireplane + 'plane_nu_AE.h5')

In [17]:
model_AE_check = load_model('../../../archive/AutoEncoder-Current/models/model_AE_2048_no_poolingUplane_nu.h5')
mean_check = np.load('../../../archive/AutoEncoder-Current/models/mean_AE_np_U.npy')
std_check = np.load('../../../archive/AutoEncoder-Current/models/std_AE_np_U.npy')

In [18]:
nu_CC_energy = {0: '.028-5.50 MeV',
                1: '5.50-7.60 MeV',
                2: '7.60-10.0 MeV',
                3: '10.0-12.0 MeV',
                4: '12.0-15.0 MeV',
                5: '15.0-17.0 MeV',
                6: '17.0-20.0 MeV',
                7: '20.0-24.0 MeV',
                8: '24.0-29.0 MeV',
                9: '29.0-85.0 MeV'
                }

nu_ES_energy = {0: '0.005-0.010 GeV',
                1: '0.010-0.013 GeV',
                2: '0.013-0.016 GeV',
                3: '0.016-0.019 GeV',
                4: '0.019-0.021 GeV',
                5: '0.021-0.024 GeV',
                6: '0.024-0.027 GeV',
                7: '0.027-0.031 GeV',
                8: '0.031-0.036 GeV',
                9: '0.036-0.079 GeV',

}


In [19]:
def make_single_pdf(x, y, predicted, interaction,energy, energy_range, wave_idx, pg_num):

    fig, axs = plt.subplots(3,2, figsize=(20, 12), facecolor='w', edgecolor='k')
    fig.subplots_adjust(hspace = .375, wspace=.1)

    axes = axs.ravel()

    for i in range(6):
        index_ = i + wave_idx
        wave_idx = index_
        axes[i].set_title(interaction + ': ' + energy_range + ' --- (peak adc: ' + str(max(y[wave_idx])) + ')')
        axes[i].plot(x[wave_idx], color='black', alpha=0.3, label='input')
        axes[i].plot(y[wave_idx], color='blue', label='target')
        axes[i].plot(predicted[wave_idx], color='m', label='prediction')
        axes[i].legend(fontsize=12)
    


    plt.savefig('./plots/tmp/tmp' +str(pg_num) + '.pdf',
                dpi=300,
                bbox_inches='tight', pad_inches=0.75)
    plt.close()

    return wave_idx

# creates and merges pdf, removes all single page pdfs from tmp folder
def make_complete_pdf(x, y, predicted, interaction, energy, energy_range, num_pages):
    wave_idx_ = 0
    page_num = 0

    while page_num < num_pages:
        wave_idx_ = make_single_pdf(x, y, predicted, interaction, energy, energy_range, wave_idx_, page_num) + 1
        page_num += 1

    merger = PdfMerger()
    path = './plots/tmp/'
    pdf_files = [path+f for f in listdir(path) if (isfile(join(path, f)))]
    print(pdf_files)
    for pdf_file in pdf_files:
        #Append PDF files
        merger.append(pdf_file)
    #merger.write('pdfs/plts_tmp/plts_' + wireplane + '_cnn_'+str(int(min_cnn*100)) + '-' + str(int(max_cnn*100)) + '_' + str(num_pages) +  'pages.pdf')
    merger.write('./plots/plt_plots'+str(energy)+'.pdf')
    merger.close()

    for file in pdf_files:
        os.remove(file)

In [21]:
def roi_ae(data_set, energy, energy_range,interaction, roi_model, ae_model, roi_scalers, ae_scalers):
    waveform_noisy = data_set[energy][0]
    print('aa: ', waveform_noisy.shape)
    noisy_wave_scaled_ROI = (waveform_noisy-roi_scalers[0])/roi_scalers[1]
    noisy_wave_scaled_AE = (waveform_noisy-ae_scalers[0])/ae_scalers[1]
    
    waveform_clean = data_set[energy][1]
    clean_wave_scaled = (waveform_clean-ae_scalers[0])/ae_scalers[1]
    counter = 0

    noisy = np.empty((0, 200))
    clean = np.empty((0, 200))
    predicted = np.empty((0, 200))

    for i in trange(len(waveform_noisy)):
        wave_roi = noisy_wave_scaled_ROI[i:i+1]
        if roi_model.predict(wave_roi, verbose=0) > 0.999:
            wave_AE = noisy_wave_scaled_AE[i:i+1]
            if max(waveform_clean[i:i+1][0]) < 1000:
                ae_pred = ae_model.predict(wave_AE, verbose=0)
                ae_pred = ae_pred.reshape(ae_pred.shape[0], ae_pred.shape[1])
                pred = ae_pred*ae_scalers[1] + ae_scalers[0]
                
                noisy_wf = waveform_noisy[i]
                noisy_wf = noisy_wf.reshape(1, 200)
                noisy = np.concatenate((noisy, noisy_wf))
                #print('debug: ', waveform_clean[i:i+1].shape, clean.shape)
                clean = np.concatenate((clean, waveform_clean[i:i+1]))
                #print('debug: ', pred.shape, predicted.shape)
                predicted = np.concatenate((predicted, pred))
    for j in range(10):
        noisy, clean, predicted = shuffle(noisy, clean, predicted)
    
    
    make_complete_pdf(noisy, clean, predicted, interaction, energy, energy_range, 20)
    
    

In [22]:
for energy_ in range(10):
    roi_ae(all_waveforms_nu_ES, energy_, nu_ES_energy[energy_], 'nuES', model_60k, model_AE_check, [model_60k_mean, model_60k_std], [mean_check, std_check])

aa:  (7080, 200)


100%|██████████| 7080/7080 [06:58<00:00, 16.90it/s]


['./plots/tmp/tmp11.pdf', './plots/tmp/tmp1.pdf', './plots/tmp/tmp2.pdf', './plots/tmp/tmp3.pdf', './plots/tmp/tmp15.pdf', './plots/tmp/tmp0.pdf', './plots/tmp/tmp13.pdf', './plots/tmp/tmp4.pdf', './plots/tmp/tmp18.pdf', './plots/tmp/tmp19.pdf', './plots/tmp/tmp16.pdf', './plots/tmp/tmp9.pdf', './plots/tmp/tmp5.pdf', './plots/tmp/tmp17.pdf', './plots/tmp/tmp12.pdf', './plots/tmp/tmp14.pdf', './plots/tmp/tmp8.pdf', './plots/tmp/tmp7.pdf', './plots/tmp/tmp6.pdf', './plots/tmp/tmp10.pdf']
aa:  (6475, 200)


100%|██████████| 6475/6475 [06:28<00:00, 16.67it/s]


In [ ]:
def roi_ae(data_set, energy, energy_range,interation, roi_model, ae_model, roi_scalers, ae_scalers, num_pred):
    waveform_noisy = data_set[energy][0]
    noisy_wave_scaled_ROI = (waveform_noisy-roi_scalers[0])/roi_scalers[1]
    noisy_wave_scaled_AE = (waveform_noisy-ae_scalers[0])/ae_scalers[1]
    
    waveform_clean = data_set[energy][1]
    clean_wave_scaled = (waveform_clean-ae_scalers[0])/ae_scalers[1]
    counter = 0
    for i in range(100000):
        wave_roi = noisy_wave_scaled_ROI[i:i+1]
        if roi_model.predict(wave_roi, verbose=0) > 0.999:
            wave_AE = noisy_wave_scaled_AE[i:i+1]
            if max(waveform_clean[i:i+1][0]) < 1000:
                ae_pred = ae_model(wave_AE)
                pred = ae_pred*ae_scalers[1] + ae_scalers[0]
                print(max(waveform_clean[i:i+1][0]))
                
                fig = plt.figure(figsize=(8,3))
                plt.title(interation + ': ' + energy_range + ' --- (peak adc: ' + str(max(waveform_clean[i:i+1][0])) + ')')
                plt.plot(pred[0], color='m', label='pred')
                plt.plot(waveform_clean[i:i+1][0], color='blue', label='target')
                plt.ylabel('adc count')
                plt.legend()
                plt.show()
            
                counter += 1
            
            if counter >= num_pred:
                break